# Module 5 Homework

In this homework we'll put what we learned about Spark in practice.

For this homework we will be using the Yellow 2024-10 data from the official website: 

```bash
wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
```


## Question 1: Install Spark and PySpark

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

> [!NOTE]
> To install PySpark follow this [guide](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/05-batch/setup/pyspark.md)

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/09 18:13:53 WARN Utils: Your hostname, DESKTOP-LFON7EH resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/09 18:13:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/09 18:13:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/09 18:13:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.version

'3.3.2'

## Question 2: Yellow October 2024

Read the October 2024 Yellow into a Spark Dataframe.

Repartition the Dataframe to 4 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 6MB
- **25MB**
- 75MB
- 100MB

In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-09 18:13:57--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 143.204.102.123, 143.204.102.231, 143.204.102.120, ...
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|143.204.102.123|:443... 
200 OKequest sent, awaiting response... 
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  29.5MB/s    in 2.1s    

2025-03-09 18:13:59 (29.5 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [5]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

In [6]:
df = df.repartition(4)

In [7]:
df.write.parquet('data/pq/yellow/2024/10/')

## Question 3: Count records 

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

- 85,567
- 105,567
- **125,567**
- 145,567

In [8]:
from pyspark.sql import functions as F

In [9]:
df \
    .withColumn('tpep_pickup_datetime', F.to_date(df.tpep_pickup_datetime)) \
    .filter("tpep_pickup_datetime = '2024-10-15'") \
    .count()

125567

## Question 4: Longest trip

What is the length of the longest trip in the dataset in hours?

- 122
- 142
- **162**
- 182

In [10]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [11]:
df \
    .withColumn('duration', (df.tpep_dropoff_datetime.cast('long') - df.tpep_pickup_datetime.cast('long')) / 3600) \
    .withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

[Stage 12:>                                                         (0 + 4) / 4]

+-----------+------------------+
|pickup_date|     max(duration)|
+-----------+------------------+
| 2024-10-16|162.61777777777777|
| 2024-10-03|           143.325|
| 2024-10-22|137.76055555555556|
| 2024-10-18|114.83472222222223|
| 2024-10-21| 89.89833333333333|
+-----------+------------------+



## Question 5: User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

- 80
- 443
- **4040**
- 8080


## Question 6: Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark:

```bash
wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
```

Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

- **Governor's Island/Ellis Island/Liberty Island**
- Arden Heights
- Rikers Island
- Jamaica Bay


In [12]:
df_zones = spark.read.csv('data/taxi_zone_lookup.csv', header=True)

In [13]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [14]:
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [15]:
df_zones.createOrReplaceTempView('zones')
df.createOrReplaceTempView('yellow_trip')

In [16]:
spark.sql("""
SELECT
    pul.Zone,
    COUNT(1)
FROM 
    yellow_trip yellow JOIN zones pul ON yellow.PULocationID = pul.LocationID
GROUP BY 
    1
ORDER BY
    2 ASC
LIMIT 5;
""").show()

[Stage 21:>                                                         (0 + 4) / 4]

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|Governor's Island...|       1|
|       Rikers Island|       2|
|       Arden Heights|       2|
|         Jamaica Bay|       3|
| Green-Wood Cemetery|       3|
+--------------------+--------+





## Submitting the solutions

- Form for submitting: https://courses.datatalks.club/de-zoomcamp-2025/homework/hw5
- Deadline: See the website